<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Feature Engineering</h4>
    <p style="font-size: 20px;">NBA API Data (2022-2024)</p>
</div>

<a name="Feature-Engineering"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

**[1. Create Team Matchups and Targets](#1.-Create-Team-Matchups-and-Targets)**

- [1.1. Clean Game Data](#1.1.-Clean-Game-Data)

- [1.2. Reshape to Game Matchups](#1.2.-Reshape-to-Game-Matchups)

- [1.3. Create Target Variables](#1.3.-Create-Target-Variables)

**[2. Create Rolling Window Statistics](#2.-Create-Rolling-Window-Statistics)**

# Setup

[Return to top](#Feature-Engineering)

In [1]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

# Data

[Return to top](#Feature-Engineering)

In [2]:
track_stats_df = pd.read_csv('../../data/original/nba_track_statistics_2021_2024.csv')

track_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT
0,22101221,1610612737,Hawks,ATL,Atlanta,240:00,16.86,27,52,77,347,2,1,228,29,13,30,0.433,32,57,0.561,0.517,16,24,0.667
1,22101221,1610612745,Rockets,HOU,Houston,240:00,17.14,20,53,69,392,4,2,274,24,15,26,0.577,26,63,0.413,0.461,16,29,0.552
2,22101207,1610612737,Hawks,ATL,Atlanta,240:00,17.36,25,40,61,377,3,2,258,17,20,30,0.667,22,58,0.379,0.477,16,18,0.889
3,22101207,1610612748,Heat,MIA,Miami,240:00,17.67,11,56,65,359,1,3,247,20,14,21,0.667,26,51,0.510,0.556,13,21,0.619
4,22101192,1610612764,Wizards,WAS,Washington,240:00,18.05,16,56,71,418,3,2,301,26,20,31,0.645,22,55,0.400,0.488,16,25,0.640


In [3]:
box_score_df = pd.read_csv('../../data/original/nba_games_box_scores_2022_2024.csv')

In [4]:
box_score_df.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7520,22023,1610612764,WAS,Washington Wizards,22300642,2024-01-27,WAS @ DET,W,240,118,45,100,0.450,11,34.0,0.324,17,21,0.810,16.0,34.0,50.0,26,10.0,4,9,19,14.0
7521,22023,1610612764,WAS,Washington Wizards,22300665,2024-01-29,WAS @ SAS,W,240,118,46,86,0.535,9,25.0,0.360,17,24,0.708,14.0,31.0,45.0,32,9.0,8,18,15,5.0
7522,22023,1610612764,WAS,Washington Wizards,22300676,2024-01-31,WAS vs. LAC,L,239,109,45,97,0.464,9,29.0,0.310,10,15,0.667,12.0,33.0,45.0,19,4.0,10,13,19,-16.0
7523,22023,1610612764,WAS,Washington Wizards,22300689,2024-02-02,WAS vs. MIA,L,239,102,37,90,0.411,11,42.0,0.262,17,21,0.810,6.0,37.0,43.0,28,5.0,4,8,25,-8.0
7524,22023,1610612764,WAS,Washington Wizards,22300705,2024-02-04,WAS vs. PHX,L,240,112,47,96,0.490,7,32.0,0.219,11,17,0.647,13.0,22.0,35.0,32,11.0,4,18,19,-28.0


In [9]:
#formats the minutes field
track_stats_df['MIN'] = track_stats_df['MIN'].str.slice(0, 3)
track_stats_df = track_stats_df.drop(track_stats_df[track_stats_df['MIN'] == '0:0'].index)
track_stats_df = track_stats_df.drop(track_stats_df[track_stats_df['MIN'] == '80:'].index)
track_stats_df['MIN'] = track_stats_df['MIN'].astype(int)
#creates team name field to match games_df
track_stats_df['TEAM_NAME'] = track_stats_df['TEAM_CITY'] + " " + track_stats_df['TEAM_NAME']
track_stats_df.drop(['TEAM_CITY'], inplace=True, axis=1)
track_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT
0,22101221,1610612737,Atlanta Hawks,ATL,240,16.86,27,52,77,347,2,1,228,29,13,30,0.433,32,57,0.561,0.517,16,24,0.667
1,22101221,1610612745,Houston Rockets,HOU,240,17.14,20,53,69,392,4,2,274,24,15,26,0.577,26,63,0.413,0.461,16,29,0.552
2,22101207,1610612737,Atlanta Hawks,ATL,240,17.36,25,40,61,377,3,2,258,17,20,30,0.667,22,58,0.379,0.477,16,18,0.889
3,22101207,1610612748,Miami Heat,MIA,240,17.67,11,56,65,359,1,3,247,20,14,21,0.667,26,51,0.510,0.556,13,21,0.619
4,22101192,1610612764,Washington Wizards,WAS,240,18.05,16,56,71,418,3,2,301,26,20,31,0.645,22,55,0.400,0.488,16,25,0.640


In [10]:
track_stats_df = pd.merge(track_stats_df, box_score_df[['SEASON_ID','GAME_DATE','MATCHUP', 'GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

track_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT,SEASON_ID,GAME_DATE,MATCHUP
0,22101221,1610612737,Atlanta Hawks,ATL,240,16.86,27,52,77,347,2,1,228,29,13,30,0.433,32,57,0.561,0.517,16,24,0.667,22021,2022-04-10,ATL @ HOU
1,22101221,1610612745,Houston Rockets,HOU,240,17.14,20,53,69,392,4,2,274,24,15,26,0.577,26,63,0.413,0.461,16,29,0.552,22021,2022-04-10,HOU vs. ATL
2,22101207,1610612737,Atlanta Hawks,ATL,240,17.36,25,40,61,377,3,2,258,17,20,30,0.667,22,58,0.379,0.477,16,18,0.889,22021,2022-04-08,ATL @ MIA
3,22101207,1610612748,Miami Heat,MIA,240,17.67,11,56,65,359,1,3,247,20,14,21,0.667,26,51,0.510,0.556,13,21,0.619,22021,2022-04-08,MIA vs. ATL
4,22101192,1610612764,Washington Wizards,WAS,240,18.05,16,56,71,418,3,2,301,26,20,31,0.645,22,55,0.400,0.488,16,25,0.640,22021,2022-04-06,WAS @ ATL


<a name="1.-Create-Team-Matchups-and-Targets"></a>
# 1. Create Team Matchups and Targets

[Return to top](#Feature-Engineering)

<a name="1.1.-Clean-Game-Data"></a>
## 1.1. Clean Game Data

[Return to top](#Feature-Engineering)

We need to do three key things to clean the data:

1. Remove games with team aggregated game times of less than 238 minutes (which will remove exhibition matches).
2. Retain only games that are part of the regular season.
3. Remove any orphans (i.e., game IDs that do not have a partner) when reshaping to matchups.

Last 3 NBA regular seasons start and end dates:

- 2021-22 season: 2021-10-19 to 2022-04-10
- 2022-23 season: 2022-10-18 to 2023-04-09
- 2023-24 season: 2023-10-24 to 2024-04-14

In [11]:
# last 3 seasons start and end dates and labels
season_start_dates = ['2021-10-19', '2022-10-18', '2023-10-24']
season_end_dates   = ['2022-04-10', '2023-04-09', '2024-04-14']
season_labels      = ['2021-22', '2022-23', '2023-24']

In [13]:
# clean up the data
track_stats_df_cleaned = utl.clean_team_bs_data(track_stats_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 739 games


In [14]:
# clean up the data
box_score_df_cleaned = utl.clean_team_bs_data(box_score_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 736 games


In [15]:
#get GAME_DATE, MATCHUP, GAME_ID, TEAM_ABBREVIATION fields from games_df
track_stats_df_cleaned = pd.merge(track_stats_df_cleaned, box_score_df_cleaned[['WL','PTS','GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION', 'PLUS_MINUS']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

track_stats_df_cleaned.sort_values(by=['PTS'])

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT,SEASON_ID,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS
889,22100075,1610612742,Dallas Mavericks,DAL,240,17.37,19,49,66,378,1,4,263,14,10,25,0.400,13,53,0.245,0.295,23,28,0.821,2021-22,2021-10-29,DAL @ DEN,L,75,-31.0
224,22100717,1610612758,Sacramento Kings,SAC,240,18.05,37,54,90,387,0,2,266,15,11,36,0.306,18,59,0.305,0.305,25,36,0.694,2021-22,2022-01-25,SAC @ BOS,L,75,-53.0
242,22100595,1610612752,New York Knicks,NYK,240,16.82,25,44,67,355,0,1,247,13,11,27,0.407,18,52,0.346,0.367,14,18,0.778,2021-22,2022-01-08,NYK @ BOS,L,75,-24.0
787,22100988,1610612742,Dallas Mavericks,DAL,240,17.30,31,48,77,396,3,2,280,11,9,25,0.360,18,61,0.295,0.314,18,22,0.818,2021-22,2022-03-09,DAL vs. NYK,L,77,-30.0
727,22100257,1610612741,Chicago Bulls,CHI,240,18.15,23,60,82,360,2,5,249,13,12,33,0.364,19,52,0.365,0.365,19,29,0.655,2021-22,2021-11-22,CHI vs. IND,L,77,-32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316,22100899,1610612759,San Antonio Spurs,SAS,290,22.67,34,59,90,479,1,4,332,35,30,51,0.588,27,54,0.500,0.543,26,35,0.743,2021-22,2022-02-25,SAS @ WAS,W,157,4.0
3480,22201230,1610612744,Golden State Warriors,GSW,240,18.08,23,70,89,481,8,1,357,47,19,29,0.655,39,67,0.582,0.604,20,28,0.714,2022-23,2023-04-09,GSW @ POR,W,157,56.0
2048,22100723,1610612766,Charlotte Hornets,CHA,240,18.13,25,48,70,403,3,3,280,37,22,33,0.667,32,57,0.561,0.581,26,34,0.765,2021-22,2022-01-26,CHA @ IND,W,158,32.0
3751,22200902,1610612746,LA Clippers,LAC,290,21.60,17,58,70,525,7,1,381,39,20,31,0.645,38,65,0.585,0.602,35,48,0.729,2022-23,2023-02-24,LAC vs. SAC,L,175,-1.0


<a name="1.2.-Reshape-to-Game-Matchups"></a>
## 1.2. Reshape to Game Matchups

[Return to top](#Feature-Engineering)

In [16]:
# identify non-stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP']

# reshape team box score data to wide format so each row is a game matchup
track_stats_matchups_df = utl.reshape_team_bs_to_matchups(track_stats_df_cleaned, non_stats_cols)

Season 2021-22: 1222 games
Season 2022-23: 1221 games
Season 2023-24: 728 games


In [17]:
track_stats_matchups_df.head()

,GAME_ID,HOME_TEAM_ID,HOME_TEAM_NAME,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_DIST,HOME_ORBC,HOME_DRBC,HOME_RBC,HOME_TCHS,HOME_SAST,HOME_FTAST,HOME_PASS,HOME_AST,HOME_CFGM,HOME_CFGA,HOME_CFG_PCT,HOME_UFGM,HOME_UFGA,HOME_UFG_PCT,HOME_FG_PCT,HOME_DFGM,HOME_DFGA,HOME_DFG_PCT,SEASON_ID,...,AWAY_TEAM_NAME,AWAY_TEAM_ABBREVIATION,AWAY_MIN,AWAY_DIST,AWAY_ORBC,AWAY_DRBC,AWAY_RBC,AWAY_TCHS,AWAY_SAST,AWAY_FTAST,AWAY_PASS,AWAY_AST,AWAY_CFGM,AWAY_CFGA,AWAY_CFG_PCT,AWAY_UFGM,AWAY_UFGA,AWAY_UFG_PCT,AWAY_FG_PCT,AWAY_DFGM,AWAY_DFGA,AWAY_DFG_PCT,AWAY_WL,AWAY_PTS,AWAY_PLUS_MINUS
0,22101221,1610612745,Houston Rockets,HOU,240,17.14,20,53,69,392,4,2,274,24,15,26,0.577,26,63,0.413,0.461,16,29,0.552,2021-22,...,Atlanta Hawks,ATL,240,16.86,27,52,77,347,2,1,228,29,13,30,0.433,32,57,0.561,0.517,16,24,0.667,W,130,16.0
1,22101207,1610612748,Miami Heat,MIA,240,17.67,11,56,65,359,1,3,247,20,14,21,0.667,26,51,0.510,0.556,13,21,0.619,2021-22,...,Atlanta Hawks,ATL,240,17.36,25,40,61,377,3,2,258,17,20,30,0.667,22,58,0.379,0.477,16,18,0.889,L,109,-4.0
2,22101192,1610612737,Atlanta Hawks,ATL,240,17.38,14,50,64,359,1,1,244,26,13,25,0.520,30,67,0.448,0.467,18,22,0.818,2021-22,...,Washington Wizards,WAS,240,18.05,16,56,71,418,3,2,301,26,20,31,0.645,22,55,0.400,0.488,16,25,0.640,L,103,-15.0
3,22101182,1610612761,Toronto Raptors,TOR,240,17.34,46,66,109,363,1,2,237,20,24,43,0.558,20,58,0.345,0.436,16,25,0.640,2021-22,...,Atlanta Hawks,ATL,240,17.33,26,59,82,366,4,2,248,31,17,33,0.515,26,61,0.426,0.457,17,24,0.708,L,108,-10.0
4,22101163,1610612737,Atlanta Hawks,ATL,240,17.81,24,63,85,357,0,5,234,18,14,28,0.500,22,58,0.379,0.419,11,20,0.550,2021-22,...,Brooklyn Nets,BKN,240,17.88,29,63,87,369,3,2,239,17,18,41,0.439,24,58,0.414,0.424,13,21,0.619,L,115,-7.0


<a name="1.3.-Create-Target-Variables"></a>
## 1.3. Create Target Variables

[Return to top](#Feature-Engineering)

There are three targets of interest:

1. **Total Game Points (over / under):** This can be calculated as the sum of `HOME_PTS + AWAY_PTS`.
2. **Difference in Game Points (plus / minus):** This can be calculated in relation to the home team as the following difference: `HOME_PTS - AWAY_PTS`.
3. **Game Winner (moneyline):** This can be defined in relation to the home team using the `HOME_WL` column, where a win for the home team is equal to 1 and a loss for the home team equal to 0. We will create a new column called `GAME_RESULT` for this indicator.

In [18]:
# create the above three target variables
track_stats_matchups_df = utl.create_target_variables(track_stats_matchups_df, 'HOME_WL', 'HOME_PTS', 'AWAY_PTS')

In [19]:
track_stats_matchups_df[['GAME_DATE', 'GAME_ID',  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_PTS', 'AWAY_PTS', 'GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS']].tail()

,GAME_DATE,GAME_ID,HOME_TEAM_NAME,AWAY_TEAM_NAME,HOME_PTS,AWAY_PTS,GAME_RESULT,TOTAL_PTS,PLUS_MINUS
3166,2023-11-22,22300225,Charlotte Hornets,Washington Wizards,117,114,1,231,3.0
3167,2023-11-10,22300009,Washington Wizards,Charlotte Hornets,117,124,0,241,-7.0
3168,2023-11-08,22300157,Charlotte Hornets,Washington Wizards,116,132,0,248,-16.0
3169,2024-01-24,22300619,Detroit Pistons,Charlotte Hornets,113,106,1,219,7.0
3170,2023-10-27,22300077,Charlotte Hornets,Detroit Pistons,99,111,0,210,-12.0


<a name="2.-Create-Rolling-Window-Statistics"></a>
# 2. Create Rolling Window Statistics

[Return to top](#Feature-Engineering)

Here we create average box scores for each team over a rolling window of the previous $n$-games.

In [20]:
# identify stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM_ID', 'AWAY_TEAM_ID',
                  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_WL', 'AWAY_WL', 'HOME_MIN', 
                  'AWAY_MIN', 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION']
stats_cols = [col for col in track_stats_matchups_df.columns if col not in non_stats_cols]

In [40]:
# calculate rolling averages for each statistic and add them to the DataFrame
track_stats_matchups_roll_df = utl.process_rolling_stats(
    track_stats_matchups_df, 
    stats_cols, 
    target_cols=['GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS'],
    window_size=5,   # the number of games to include in the rolling window
    min_obs=1,       # the minimum number of observations present within the window to yield an aggregate value
    stratify_by_season=True,  # should the rolling calculations be reset at the start of each new season or be contiguous across seasons? 
    exclude_initial_games=0   # number of initial games to exclude from the rolling averages (optionally by season)
)

In [41]:
track_stats_matchups_roll_df.tail()

,GAME_ID,GAME_RESULT,TOTAL_PTS,PLUS_MINUS,HOME_TEAM_NAME,SEASON_ID,GAME_DATE,ROLL_HOME_DIST,ROLL_HOME_ORBC,ROLL_HOME_DRBC,ROLL_HOME_RBC,ROLL_HOME_TCHS,ROLL_HOME_SAST,ROLL_HOME_FTAST,ROLL_HOME_PASS,ROLL_HOME_AST,ROLL_HOME_CFGM,ROLL_HOME_CFGA,ROLL_HOME_CFG_PCT,ROLL_HOME_UFGM,ROLL_HOME_UFGA,ROLL_HOME_UFG_PCT,ROLL_HOME_FG_PCT,ROLL_HOME_DFGM,ROLL_HOME_DFGA,ROLL_HOME_DFG_PCT,ROLL_HOME_PTS,AWAY_TEAM_NAME,ROLL_AWAY_DIST,ROLL_AWAY_ORBC,ROLL_AWAY_DRBC,ROLL_AWAY_RBC,ROLL_AWAY_TCHS,ROLL_AWAY_SAST,ROLL_AWAY_FTAST,ROLL_AWAY_PASS,ROLL_AWAY_AST,ROLL_AWAY_CFGM,ROLL_AWAY_CFGA,ROLL_AWAY_CFG_PCT,ROLL_AWAY_UFGM,ROLL_AWAY_UFGA,ROLL_AWAY_UFG_PCT,ROLL_AWAY_FG_PCT,ROLL_AWAY_DFGM,ROLL_AWAY_DFGA,ROLL_AWAY_DFG_PCT,ROLL_AWAY_PTS
2540,22300703,0,218,-16.0,San Antonio Spurs,2023-24,2024-02-03,19.229,23.00,53.25,74.00,413.15,3.90,2.60,289.55,30.30,18.85,36.65,0.512,23.25,53.85,0.432,0.465,20.90,32.35,0.647,114.60,Cleveland Cavaliers,18.206,23.50,57.75,78.60,406.20,2.90,2.70,285.50,25.30,18.85,38.10,0.498,22.20,48.05,0.461,0.476,17.05,27.05,0.642,112.80
3077,22300705,0,252,-28.0,Washington Wizards,2023-24,2024-02-04,18.500,21.20,54.55,72.40,406.50,4.60,2.45,285.65,29.65,19.50,35.90,0.549,24.05,53.90,0.446,0.487,22.50,34.05,0.668,115.10,Phoenix Suns,17.704,23.30,55.85,77.45,407.75,5.55,1.90,285.10,27.50,19.70,36.95,0.541,24.35,49.00,0.498,0.515,17.55,28.45,0.628,119.85
3017,22300704,0,210,-12.0,Detroit Pistons,2023-24,2024-02-04,18.238,23.75,53.15,74.90,386.60,3.05,2.80,265.60,26.75,18.30,36.80,0.500,23.45,51.50,0.456,0.472,20.25,30.30,0.668,113.05,Orlando Magic,18.541,27.40,54.25,79.10,395.45,1.80,2.95,274.35,23.95,18.60,36.90,0.501,21.90,49.60,0.441,0.469,18.90,28.50,0.670,111.70
3036,22300707,0,214,-18.6,Charlotte Hornets,2023-24,2024-02-04,18.740,22.20,54.20,74.80,394.25,2.60,2.70,275.45,24.05,17.15,35.60,0.490,22.65,51.95,0.436,0.455,19.60,28.60,0.686,107.55,Indiana Pacers,19.101,28.85,51.50,77.75,430.35,3.70,3.05,305.60,30.35,18.95,36.00,0.523,27.35,57.20,0.483,0.500,21.40,33.00,0.653,121.90
2492,22300706,1,222,40.0,Boston Celtics,2023-24,2024-02-04,17.714,24.35,60.95,83.25,384.50,3.80,1.75,265.15,25.35,17.20,32.25,0.530,26.05,56.80,0.459,0.486,17.05,28.50,0.602,121.05,Memphis Grizzlies,18.582,25.35,58.70,81.60,404.75,3.80,1.60,284.55,25.00,17.45,35.45,0.490,21.90,51.10,0.428,0.453,19.45,31.80,0.610,110.10


In [42]:
# write out the matchups with rolling features
track_stats_matchups_roll_df.to_csv('../../data/processed/nba_team_matchups_rolling_player_track_box_stats_2021_2024_r05.csv', index=False)